In [9]:
import pandas as pd
import os
import re
from datetime import datetime

# Função para parsear um único arquivo de log (já fornecida anteriormente)
def parse_log_to_dataframe(log_file_path):
    # Listas para armazenar os dados
    experiment_config = {}
    batch_results = []
    save_model_result = {}
    inference_result = {}
    
    # Variáveis para rastrear o estado
    current_section = None
    epoch = 0
    
    # Regex para extrair informações
    config_pattern = r"(\w+)\s*:\s*(.+)"
    result_pattern = r"(\w+\s*\w*\s*\w*)\s*:\s*([-]?\d+\.\d+|\d+\.\d+|-?\d+)"
    
    try:
        with open(log_file_path, 'r') as file:
            for line in file:
                line = line.strip()
                
                # Identificar seção
                if "EXPERIMENT CONFIGURATION" in line:
                    current_section = "config"
                    continue
                elif "EXPERIMENT RESULTS" in line:
                    if len(batch_results) < 2:
                        current_section = "batch"
                        batch_results.append({})
                    elif len(batch_results) == 2 and not save_model_result:
                        current_section = "save_model"
                    else:
                        current_section = "inference"
                    continue
                elif "Época" in line and "concluída" in line:
                    epoch += 1
                    continue
                
                # Extrair configuração
                if current_section == "config":
                    match = re.match(config_pattern, line.split(" - INFO - ")[-1])
                    if match:
                        key, value = match.groups()
                        experiment_config[key.strip()] = value.strip()
                
                # Extrair resultados
                if current_section in ["batch", "save_model", "inference"]:
                    match = re.match(result_pattern, line.split(" - INFO - ")[-1])
                    if match:
                        key, value = match.groups()
                        key = key.strip()
                        value = float(value)
                        
                        if current_section == "batch":
                            batch_results[-1][key] = value
                            batch_results[-1]["Epoch"] = epoch
                        elif current_section == "save_model":
                            save_model_result[key] = value
                        elif current_section == "inference":
                            inference_result[key] = value
    
        # Criar DataFrame para resultados de batches
        batch_df = pd.DataFrame(batch_results)
        batch_df["Phase"] = "Training"
        
        # Criar DataFrame para salvar modelo
        save_model_df = pd.DataFrame([save_model_result])
        save_model_df["Phase"] = "Save Model"
        save_model_df["Epoch"] = None
        
        # Criar DataFrame para inferência
        inference_df = pd.DataFrame([inference_result])
        inference_df["Phase"] = "Inference"
        inference_df["Epoch"] = None
        
        # Combinar todos os DataFrames
        df = pd.concat([batch_df, save_model_df, inference_df], ignore_index=True)
        
        # Adicionar informações de configuração como colunas
        for key, value in experiment_config.items():
            df[key] = value
        
        # Reorganizar colunas
        columns = ["Phase", "Epoch", "Elapsed Time Seconds", "Cpu Usage Percent", 
                   "Ram Usage Bytes", "Vram Usage Bytes", "Accuracy", "Precision", 
                   "Recall", "F1 Score", "model_name", "config_path", 
                   "train_data_path", "val_data_path", "model_params", 
                   "train_params", "inference_params"]
        
        # Garantir que todas as colunas existam
        for col in columns:
            if col not in df.columns:
                df[col] = None
        
        df = df[columns]
        
        return df
    
    except Exception as e:
        print(f"Erro ao processar {log_file_path}: {str(e)}")
        return None

# Função para salvar um DataFrame em CSV (já fornecida anteriormente)
def save_log_to_csv(log_file_path, output_base_dir="logs_csv"):
    # Obter o nome do arquivo sem a extensão
    file_name = os.path.splitext(os.path.basename(log_file_path))[0]
    
    # Criar o diretório de saída específico para o arquivo
    output_dir = os.path.join(output_base_dir, file_name)
    os.makedirs(output_dir, exist_ok=True)
    
    # Gerar o DataFrame a partir do log
    df = parse_log_to_dataframe(log_file_path)
    
    if df is not None:
        # Caminho do arquivo CSV
        csv_file_path = os.path.join(output_dir, f"{file_name}.csv")
        
        # Salvar o DataFrame como CSV
        df.to_csv(csv_file_path, index=False)
        print(f"CSV salvo em: {csv_file_path}")
        return csv_file_path
    else:
        print(f"Falha ao gerar CSV para {log_file_path}")
        return None

# Nova função para processar todos os arquivos na pasta logs/
def process_all_logs(logs_dir="logs", output_base_dir="logs_csv"):
    # Verificar se o diretório de logs existe
    if not os.path.exists(logs_dir):
        print(f"Diretório {logs_dir} não encontrado.")
        return []
    
    # Listar todos os arquivos .log no diretório
    log_files = [f for f in os.listdir(logs_dir) if f.endswith(".log")]
    
    if not log_files:
        print(f"Nenhum arquivo .log encontrado em {logs_dir}.")
        return []
    
    # Lista para armazenar os caminhos dos CSVs gerados
    csv_paths = []
    
    # Processar cada arquivo de log
    for log_file in log_files:
        log_file_path = os.path.join(logs_dir, log_file)
        print(f"Processando {log_file_path}...")
        csv_path = save_log_to_csv(log_file_path, output_base_dir)
        if csv_path:
            csv_paths.append(csv_path)
    
    print(f"Processamento concluído. {len(csv_paths)} arquivos CSV gerados.")
    return csv_paths

# Exemplo de uso


In [11]:
csv_paths = process_all_logs(logs_dir="logs")

Processando logs\mobilenet_v1_epochs_2_bs_8_20250930_222801.log...
CSV salvo em: logs_csv\mobilenet_v1_epochs_2_bs_8_20250930_222801\mobilenet_v1_epochs_2_bs_8_20250930_222801.csv
Processando logs\mobilenet_v1_epochs_2_bs_8_20250930_232125.log...
CSV salvo em: logs_csv\mobilenet_v1_epochs_2_bs_8_20250930_232125\mobilenet_v1_epochs_2_bs_8_20250930_232125.csv
Processamento concluído. 2 arquivos CSV gerados.
